In [ ]:
from datetime import date
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Data Selection and modify start and end date
train_howmanyyears_begin = 3
train_howmanymonths_end = 1
today = date.today()
TRAIN_START_DATE = date(today.year - train_howmanyyears_begin,today.month, today.day).isoformat()
TRAIN_END_DATE = date(today.year ,today.month - train_howmanymonths_end, today.day).isoformat()

test_howmanymonths_begin = 1
TEST_START_DATE = date(today.year ,today.month - test_howmanymonths_begin, today.day).isoformat()
TEST_END_DATE = today.isoformat()



In [ ]:
print(TRAIN_START_DATE)
print(TRAIN_END_DATE)
print(TEST_START_DATE)
print(TEST_END_DATE)

2022-03-25
2025-02-25
2025-02-25
2025-03-25


In [ ]:
# LOda the training data
whichstock = 'TATASTEEL.NS'
data_train = yf.download(whichstock,start = TRAIN_START_DATE , end = TRAIN_END_DATE)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [ ]:

print(data_train.columns)

MultiIndex([( 'Close', 'TATASTEEL.NS'),
            (  'High', 'TATASTEEL.NS'),
            (   'Low', 'TATASTEEL.NS'),
            (  'Open', 'TATASTEEL.NS'),
            ('Volume', 'TATASTEEL.NS')],
           names=['Price', 'Ticker'])


In [ ]:
data_train.head()

In [ ]:
data_train.reset_index(inplace=True)

In [ ]:
data_train['Date'] = pd.to_datetime(data_train['Date'])

In [ ]:
data_train.info()

In [ ]:
data_train.shape

(723, 6)

In [ ]:
# Load test data
data_test =  yf.download(whichstock,start = TEST_START_DATE , end = TEST_END_DATE)
data_test.reset_index(inplace=True)
data_test['Date'] = pd.to_datetime(data_test['Date'])

[*********************100%***********************]  1 of 1 completed


In [ ]:
data_test.head()

Price,Date,Close,High,Low,Open,Volume
Ticker,,TATASTEEL.NS,TATASTEEL.NS,TATASTEEL.NS,TATASTEEL.NS,TATASTEEL.NS
0,2025-02-25,137.070007,139.350006,136.699997,138.490005,23575603
1,2025-02-27,138.690002,138.990005,135.880005,137.210007,30747044
2,2025-02-28,137.199997,137.800003,134.059998,137.800003,45540594
3,2025-03-03,138.570007,139.050003,134.309998,137.000000,31154401
4,2025-03-04,139.339996,140.940002,135.809998,136.149994,28634645


In [ ]:
plt.figure(figsize = (8,6))
plt.plot(data_train['Date'],data_train['Low'],label = 'low',color = 'red')
plt.plot(data_train['Date'],data_train['High'],label = 'high',color = 'green')
plt.ylabel("Stock price in INR")
plt.xlabel("Date")
plt.title(f"High vs low of {whichstock}")
plt.xticks(rotation=90)
plt.legend();


In [ ]:
data_train['EMA-50'] = data_train['Open','TATASTEEL.NS'].ewm(span = 50, adjust=False).mean()
data_train['EMA-200'] = data_train['Open','TATASTEEL.NS'].ewm(span = 200, adjust=False).mean()

In [ ]:
data_train.head()

Price,Date,Close,High,Low,Open,Volume,EMA-50,EMA-200
Ticker,,TATASTEEL.NS,TATASTEEL.NS,TATASTEEL.NS,TATASTEEL.NS,TATASTEEL.NS,,
0,2022-03-25,56.498829,57.640241,56.270965,57.171861,69457440,57.171861,57.171861
1,2022-03-28,56.498829,57.176079,56.019896,56.543133,46554910,57.147205,57.165605
2,2022-03-29,56.317387,56.802649,55.994585,56.711922,43202370,57.130136,57.161091
3,2022-03-30,55.194962,56.355362,54.855284,56.332157,58483160,57.098842,57.152843
4,2022-03-31,55.159100,55.718200,54.859502,55.192857,40067950,57.024098,57.133341


In [ ]:
plt.figure(figsize = (8,6))
plt.plot(data_train['Date'],data_train['EMA-50'],label = 'EWMA-50',color = 'orange')
plt.plot(data_train['Date'],data_train['EMA-200'],label = 'EWMA-200',color = 'magenta')
plt.plot(data_train['Date'],data_train['Open'],label = 'Open',color = 'green')
plt.ylabel("Stock price in INR")
plt.xlabel("Date")
plt.title(f"EWMA for Stockprice of {whichstock}")
plt.xticks(rotation=90)
plt.legend();


In [ ]:
# Change column names  for easy readability
df_train = pd.DataFrame(data_train)

In [ ]:
df_train.columns

In [ ]:
df_train = df_train.set_axis(['Date','Close','High','Low','Open','Volume','EMA-50','EMA-200'],axis = 1 )

In [ ]:
df_train.columns

Index(['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'EMA-50', 'EMA-200'], dtype='object')

In [ ]:
# Change column names  for easy readability
df_test = pd.DataFrame(data_test)
df_test = df_test.set_axis(['Date','Close','High','Low','Open','Volume'],axis = 1 )

In [ ]:
df_test.columns

Index(['Date', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')

In [ ]:
# SElect Open from the columns for training
training_set = df_train.iloc[:,4:5]
training_set[0:5]

In [ ]:
# Data Preprocessing
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)

In [ ]:
print(training_set_scaled[0:5])

[[0.11173078]
 [0.10719023]
 [0.10840918]
 [0.10566659]
 [0.09743879]]


In [ ]:
# create input data with 60 timesteps and one output
n_timesteps = 50
X_train =[]
Y_train =[]

for i in range(n_timesteps, len(training_set_scaled)):
      X_train.append(training_set_scaled[i-n_timesteps: i , 0])
      Y_train.append(training_set_scaled[i,0])
X_train, Y_train = np.array(X_train), np.array(Y_train)

In [ ]:
print(X_train[1:5,])

In [ ]:
# Reshape X_train
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_train.shape

(673, 50, 1)

In [ ]:
#Building of RNN
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
n_units = 50
n_drop = 0.2
# initialize RNN
regressor = Sequential()
regressor.add(LSTM(units = n_units, return_sequences= True, input_shape = (X_train.shape[1],1)))
regressor.add(Dropout(n_drop))
regressor.add(LSTM(units = n_units, return_sequences= True))
regressor.add(Dropout(n_drop))
regressor.add(LSTM(units = n_units, return_sequences= True))
regressor.add(Dropout(n_drop))
regressor.add(LSTM(units = n_units, return_sequences= False))
regressor.add(Dropout(n_drop))
regressor.add(Dense(units = 1)) # output layer

# Compile
regressor.compile(optimizer='adam', loss='mean_squared_error')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Fitting the training data
regressor.fit(X_train, Y_train , epochs = 100 , batch_size = 60)

In [ ]:
# Making predictions and visulize results
real_stock_price = df_test.iloc[:,4:5]
df_total = pd.concat((df_train['Open'], df_test['Open']), axis = 0 )
input = df_total[len(df_total)-len(df_test)-n_timesteps:].values

In [ ]:
input = input.reshape(-1,1)

In [ ]:
input = sc.transform(input)

In [ ]:
X_test = []
for i in range(n_timesteps , n_timesteps + len(df_test)):
   X_test.append(input[i-n_timesteps: i, 0])

In [ ]:
len(X_test)

18

In [ ]:
X_test = np.array(X_test)
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

In [ ]:
X_test.shape

(18, 50, 1)

In [ ]:
predicted_stock_price = regressor.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 391ms/step


In [ ]:
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
plt.figure(figsize = (8,6))
plt.plot(real_stock_price,color = 'red', label = "Observed stock price")
plt.plot(predicted_stock_price,label = "Predicted Stock price ",color = 'blue')
plt.ylabel("Stock price in INR")
plt.xlabel("Date")
plt.title(f"Predicted vs Observed Stockprice of {whichstock}")
plt.xticks(rotation=90)
plt.legend();
